In [1]:
name = []
rate = []
review = []
sort = []
url = []
desc = []
address = []
image = []

In [127]:
driver.quit()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC #selenium에서 사용할 모듈 import
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
import time
import requests
from bs4 import BeautifulSoup
import re
import csv
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome("C:/chromedriver",options=options) #selenium 사용에 필요한 chromedriver.exe 파일 경로 지정

driver.get("https://map.naver.com/v5/") #네이버 신 지도 

time.sleep(3)

search_box = driver.find_element_by_class_name("input_search")
search_box.send_keys("경의선숲길 주변식당")
search_box.send_keys(Keys.ENTER) #검색창에 "서울 칵테일바" 입력
time.sleep(3)
frame = driver.find_element_by_css_selector("iframe#searchIframe")
driver.switch_to.frame(frame)

scroll_div = driver.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[1]")
#검색 결과로 나타나는 scroll-bar 포함한 div 잡고
driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)

page = 1
while page<6: #몇 페이지까지 크롤링할 것인지 지정
    stores_box = driver.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[1]/ul")
    stores = driver.find_elements_by_css_selector("li._1EKsQ._12tNp")
   #해당 페이지에서 표시된 모든 가게 정보
   
    names = driver.find_elements_by_css_selector("span.OXiLu")
    for i in range(len(names)):
        naming = names[i].text
        name.append(naming)
        
    for store in stores: #한 페이지 내에서의 반복문. 순차적으로 가게 정보에 접근   
#         try:    
#             rating = re.search('/span>(\d).', store.find_element_by_css_selector("span.place_blind").get_attribute('innerHTML')).groups()[0]
#             rate.append(rating)
#         except:
#             rating = ''
#             rate.append(rating)
        time.sleep(2)
       # 평점 숫자 부분만 rating에 담음. 평점이 없는 경우가 있어 예외 처리
            #_pcma_list_scroll_container > ul > li:nth-child(5) > div.zZGuI > div > a:nth-child(1) > div > div
#         try: #_pcmap_list_scroll_container > ul > li:nth-child(2) > div.zZGuI > div > a:nth-child(1) > div > div
#             img_src = re.search('url[(]"([\S]+)"', store.find_element_by_css_selector("div.zZGuI > div > a:nth-child(1) > div > div").get_attribute('style')).groups()[0]
#             image.append(img_src)
#         except:
#             img_src = ''
#             image.append(img_src)
        
        # 리뷰개수
        try:
            review_count = store.find_element_by_css_selector("div._17H46 > span:nth-child(3)")
            review.append(review_count.text)
        except NoSuchElementException as n:
            review.append('')

        #역시 대표 이미지가 없는 경우가 있어 예외 처리
        click_name = store.find_element_by_css_selector("span.OXiLu")
        driver.execute_script("arguments[0].click();", click_name)
        

        driver.switch_to.default_content()
        time.sleep(3)        
        ##오래 헤맸던 부분!! switch_to.default_content()로 전환해야 frame_in iframe을 제대로 잡을 수 있다. 
        frame_in = driver.find_element_by_xpath('/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-place-bridge/div/nm-external-frame-bridge/nm-iframe/iframe')
        driver.switch_to.frame(frame_in) 
        
        # 평점
        try:
            rates = driver.find_element_by_css_selector("span._1Y6hi._1A8_M>em")
            rate.append(rates.text)
        except NoSuchElementException as n:
            rate.append('')
            
        # 구분
        sorting = driver.find_element_by_css_selector("span._3ocDE")
        sort.append(sorting.text)

        # 설명
        try:
            descript = driver.find_element_by_css_selector("span.WoYOw")
            desc.append(descript.text)
        except NoSuchElementException as n:
            desc.append('')
        
        time.sleep(1)
        # 위치
        addresses = driver.find_element_by_css_selector("span._2yqUQ")
        address.append(addresses.text)
        
        #url
        url.append(driver.current_url)

       #이미지
        try:
            element = driver.find_element_by_css_selector('#ibu_1')
            driver.execute_script("arguments[0].click();", element)
            time.sleep(2)
            element = driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(6) > div > div.place_section_content > div > div > a:nth-child(2)')
            driver.execute_script("arguments[0].click();", element)
            time.sleep(2)
            img = driver.find_element_by_css_selector('li._2OSze>a>img')
            image.append(img.get_attribute('src'))
        except NoSuchElementException as n:
            image.append('')
            

        driver.switch_to.default_content()
        driver.switch_to.frame(frame)
        time.sleep(3)
    element = driver.find_element_by_css_selector('#app-root > div > div._2lx2y > div._2ky45 > a:nth-child(7) > svg')
    webdriver.ActionChains(driver).move_to_element(element ).click(element ).perform()
    page = page+1
    time.sleep(3)

C:\Users\으쨔\AppData\Local\Temp/ipykernel_19012/3901785361.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/chromedriver",options=options) #selenium 사용에 필요한 chromedriver.exe 파일 경로 지정
C:\Users\으쨔\AppData\Local\Temp/ipykernel_19012/3901785361.py:22: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_box = driver.find_element_by_class_name("input_search")
C:\Users\으쨔\AppData\Local\Temp/ipykernel_19012/3901785361.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  frame = driver.find_element_by_css_selector("iframe#searchIframe")
C:\Users\으쨔\AppData\Local\Temp/ipykernel_19012/3901785361.py:29: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  scroll_div = driver.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[1]")
C:\Users\으쨔\AppData\Local\Temp/ipyke

In [117]:
import pandas as pd
df = pd.DataFrame({'식당이름': name,'평점':rate,'리뷰개수':review,'분류':sort,'주소':address, '설명':desc,'url':url, '이미지':image})
df

,식당이름,평점,리뷰개수,분류,주소,설명,url,이미지
0,궁전꽃게장,4.21,방문자리뷰 136,게요리,"전북 군산시 궁포1로 24-5 세종빌딩 2층 203호, 204호",군산 롯데몰 킴스락볼링장,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
1,창성양꼬치,4.77,방문자리뷰 241,양꼬치,전북 군산시 진포로 223,- 영업시간 : 오전11시~저녁12시 - - 북경오리구 이는 오전에 예약하셔야 저녁...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
2,불타는명태찜,4.43,방문자리뷰 284,"해물,생선요리",전북 군산시 경촌2길 38,,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
3,커피틱톡,,블로그리뷰 21,카페,전북 군산시 구암3.1로 91-77 2층,이마트 주차장으로 들어오셔서 주차장내에 있는 주유소를 지나자마자 우회전하셔서 200...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
4,군산짬뽕,4.41,방문자리뷰 226,중식당,전북 군산시 조촌4길 24-9 군산짬뽕,시청 뒤 제2 공영주차장 뒷쪽에 위치해있습니다.,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
...,...,...,...,...,...,...,...,...
95,역전할머니맥주 조촌점,4.4,방문자리뷰 148,"맥주,호프",전북 군산시 궁포3로 13 디오션프라자 104호,가성비 좋고 겨울인데도 맥주집에 사람 많았어요 직원분들 친절하시고 테이블도 치워주시...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
96,지호식당,4.31,방문자리뷰 59,한식,전북 군산시 백릉로 89,천연 재료로 직접 만든 소시지와 햄으로 정성들 여 부대찌개를 만들었습니다 건강하고 ...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
97,은아반점,4.17,방문자리뷰 94,중식당,전북 군산시 조촌로 115,안녕하세요 ~ 은아반점 홈페이지에 방문해주셔 서 감사드립니다. 군산시 조촌동에 위치...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
98,유로코피자 군산점,4.38,방문자리뷰 105,피자,전북 군산시 궁포3로 14 하늘빌딩 107호 유로코피자,"군산시 궁포 3로 14, 디오션시티 푸르지오아파트 사거리 하늘빌딩(뚜레쥬르) 옆 유...",https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...


In [128]:
df['리뷰개수'] = df['리뷰개수'].str.strip('방문자리뷰')

#df 전체에 다 적용시킬 때
# df.apply(lambda x: x.str.strip(), axis = 1)

In [129]:
df

,식당이름,평점,리뷰개수,분류,주소,설명,url,이미지
0,궁전꽃게장,4.21,136,게요리,"전북 군산시 궁포1로 24-5 세종빌딩 2층 203호, 204호",군산 롯데몰 킴스락볼링장,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
1,창성양꼬치,4.77,241,양꼬치,전북 군산시 진포로 223,- 영업시간 : 오전11시~저녁12시 - - 북경오리구 이는 오전에 예약하셔야 저녁...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
2,불타는명태찜,4.43,284,"해물,생선요리",전북 군산시 경촌2길 38,NaN,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
3,커피틱톡,NaN,블로그리뷰 21,카페,전북 군산시 구암3.1로 91-77 2층,이마트 주차장으로 들어오셔서 주차장내에 있는 주유소를 지나자마자 우회전하셔서 200...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
4,군산짬뽕,4.41,226,중식당,전북 군산시 조촌4길 24-9 군산짬뽕,시청 뒤 제2 공영주차장 뒷쪽에 위치해있습니다.,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
...,...,...,...,...,...,...,...,...
95,역전할머니맥주 조촌점,4.40,148,"맥주,호프",전북 군산시 궁포3로 13 디오션프라자 104호,가성비 좋고 겨울인데도 맥주집에 사람 많았어요 직원분들 친절하시고 테이블도 치워주시...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
96,지호식당,4.31,59,한식,전북 군산시 백릉로 89,천연 재료로 직접 만든 소시지와 햄으로 정성들 여 부대찌개를 만들었습니다 건강하고 ...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
97,은아반점,4.17,94,중식당,전북 군산시 조촌로 115,안녕하세요 ~ 은아반점 홈페이지에 방문해주셔 서 감사드립니다. 군산시 조촌동에 위치...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
98,유로코피자 군산점,4.38,105,피자,전북 군산시 궁포3로 14 하늘빌딩 107호 유로코피자,"군산시 궁포 3로 14, 디오션시티 푸르지오아파트 사거리 하늘빌딩(뚜레쥬르) 옆 유...",https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...


In [130]:
query_txt = '경의선숲길'

In [131]:
df.to_csv('주변식당/{}.csv'.format(query_txt),index=False)

In [132]:
import pandas as pd
df = pd.read_csv('주변식당/경의선숲길.csv')
df

,식당이름,평점,리뷰개수,분류,주소,설명,url,이미지
0,궁전꽃게장,4.21,136,게요리,"전북 군산시 궁포1로 24-5 세종빌딩 2층 203호, 204호",군산 롯데몰 킴스락볼링장,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
1,창성양꼬치,4.77,241,양꼬치,전북 군산시 진포로 223,- 영업시간 : 오전11시~저녁12시 - - 북경오리구 이는 오전에 예약하셔야 저녁...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
2,불타는명태찜,4.43,284,"해물,생선요리",전북 군산시 경촌2길 38,NaN,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
3,커피틱톡,NaN,블로그리뷰 21,카페,전북 군산시 구암3.1로 91-77 2층,이마트 주차장으로 들어오셔서 주차장내에 있는 주유소를 지나자마자 우회전하셔서 200...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
4,군산짬뽕,4.41,226,중식당,전북 군산시 조촌4길 24-9 군산짬뽕,시청 뒤 제2 공영주차장 뒷쪽에 위치해있습니다.,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
...,...,...,...,...,...,...,...,...
95,역전할머니맥주 조촌점,4.40,148,"맥주,호프",전북 군산시 궁포3로 13 디오션프라자 104호,가성비 좋고 겨울인데도 맥주집에 사람 많았어요 직원분들 친절하시고 테이블도 치워주시...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
96,지호식당,4.31,59,한식,전북 군산시 백릉로 89,천연 재료로 직접 만든 소시지와 햄으로 정성들 여 부대찌개를 만들었습니다 건강하고 ...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
97,은아반점,4.17,94,중식당,전북 군산시 조촌로 115,안녕하세요 ~ 은아반점 홈페이지에 방문해주셔 서 감사드립니다. 군산시 조촌동에 위치...,https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...
98,유로코피자 군산점,4.38,105,피자,전북 군산시 궁포3로 14 하늘빌딩 107호 유로코피자,"군산시 궁포 3로 14, 디오션시티 푸르지오아파트 사거리 하늘빌딩(뚜레쥬르) 옆 유...",https://map.naver.com/v5/search/%EA%B2%BD%EC%9...,https://search.pstatic.net/common/?autoRotate=...


In [133]:
df['이미지'].isnull().sum()

9